# Введение в MapReduce модель на Python


In [59]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

In [60]:
def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)

def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)

Модель элемента данных

In [61]:
class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str

In [62]:
input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

Функция RECORDREADER моделирует чтение элементов с диска или по сети.

In [63]:
def RECORDREADER():
  return [(u.id, u) for u in input_collection]

In [64]:
list(RECORDREADER())

[(0, User(id=0, age=55, social_contacts=20, gender='male')),
 (1, User(id=1, age=25, social_contacts=240, gender='female')),
 (2, User(id=2, age=25, social_contacts=500, gender='female')),
 (3, User(id=3, age=33, social_contacts=800, gender='female'))]

In [65]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

In [66]:
map_output = flatten(map(lambda x: MAP(*x), RECORDREADER()))
map_output = list(map_output) # materialize
map_output

[(25, User(id=1, age=25, social_contacts=240, gender='female')),
 (25, User(id=2, age=25, social_contacts=500, gender='female')),
 (33, User(id=3, age=33, social_contacts=800, gender='female'))]

In [67]:
def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

In [68]:
shuffle_output = groupbykey(map_output)
shuffle_output = list(shuffle_output)
shuffle_output

[(25,
  [User(id=1, age=25, social_contacts=240, gender='female'),
   User(id=2, age=25, social_contacts=500, gender='female')]),
 (33, [User(id=3, age=33, social_contacts=800, gender='female')])]

In [69]:
reduce_output = flatten(map(lambda x: REDUCE(*x), shuffle_output))
reduce_output = list(reduce_output)
reduce_output

[(25, 370.0), (33, 800.0)]

Все действия одним конвейером!

In [70]:
list(flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER()))))))

[(25, 370.0), (33, 800.0)]

# **MapReduce**
Выделим общую для всех пользователей часть системы в отдельную функцию высшего порядка. Это наиболее простая модель MapReduce, без учёта распределённого хранения данных.

Пользователь для решения своей задачи реализует RECORDREADER, MAP, REDUCE.

In [71]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

## Спецификация MapReduce



```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*

mapreduce ((k1,v1)*) -> (k3,v3)*
groupby ((k2,v2)*) -> (k2,v2*)*
flatten (e2**) -> e2*

mapreduce .map(f).flatten.groupby(k2).map(g).flatten
```




# Примеры

## SQL

In [72]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str

input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)

def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)

def RECORDREADER():
  return [(u.id, u) for u in input_collection]

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(25, 370.0), (33, 800.0)]

## Matrix-Vector multiplication

In [73]:
from typing import Iterator
import numpy as np

mat = np.ones((5,4))
vec = np.random.rand(4) # in-memory vector in all map tasks

def MAP(coordinates:(int, int), value:int):
  i, j = coordinates
  yield (i, value*vec[j])

def REDUCE(i:int, products:Iterator[NamedTuple]):
  sum = 0
  for p in products:
    sum += p
  yield (i, sum)

def RECORDREADER():
  for i in range(mat.shape[0]):
    for j in range(mat.shape[1]):
      yield ((i, j), mat[i,j])

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(0, np.float64(1.9271258208095592)),
 (1, np.float64(1.9271258208095592)),
 (2, np.float64(1.9271258208095592)),
 (3, np.float64(1.9271258208095592)),
 (4, np.float64(1.9271258208095592))]

## Inverted index

In [74]:
from typing import Iterator

d1 = "it is what it is"
d2 = "what is it"
d3 = "it is a banana"
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    yield ("{}".format(docid), document)

def MAP(docId:str, body:str):
  for word in set(body.split(' ')):
    yield (word, docId)

def REDUCE(word:str, docIds:Iterator[str]):
  yield (word, sorted(docIds))

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('is', ['0', '1', '2']),
 ('what', ['0', '1']),
 ('it', ['0', '1', '2']),
 ('a', ['2']),
 ('banana', ['2'])]

## WordCount

In [75]:
from typing import Iterator

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    for (lineid, line) in enumerate(document.split('\n')):
      yield ("{}:{}".format(docid,lineid), line)

def MAP(docId:str, line:str):
  for word in line.split(" "):
    yield (word, 1)

def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('', 3), ('it', 9), ('is', 9), ('what', 5), ('a', 1), ('banana', 1)]

# MapReduce Distributed

Добавляется в модель фабрика RECORDREARER-ов --- INPUTFORMAT, функция распределения промежуточных результатов по партициям PARTITIONER, и функция COMBINER для частичной аггрегации промежуточных результатов до распределения по новым партициям.

In [76]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def groupbykey_distributed(map_partitions, PARTITIONER):
  global reducers
  partitions = [dict() for _ in range(reducers)]
  for map_partition in map_partitions:
    for (k2, v2) in map_partition:
      p = partitions[PARTITIONER(k2)]
      p[k2] = p.get(k2, []) + [v2]
  return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]

def PARTITIONER(obj):
  global reducers
  return hash(obj) % reducers

def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
  map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())
  if COMBINER != None:
    map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)
  reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER) # shuffle
  reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)

  print("{} key-value pairs were sent over a network.".format(sum([len(vs) for (k,vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])))
  return reduce_outputs

## Спецификация MapReduce Distributed


```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*

e1 (k1, v1)
e2 (k2, v2)
partition1 (k2, v2)*
partition2 (k2, v2*)*

flatmap (e1->e2*, e1*) -> partition1*
groupby (partition1*) -> partition2*

mapreduce ((k1,v1)*) -> (k3,v3)*
mapreduce .flatmap(f).groupby(k2).flatmap(g)
```



## WordCount

In [77]:
from typing import Iterator
import numpy as np

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3, d1, d2, d3]

maps = 3
reducers = 2

def INPUTFORMAT():
  global maps

  def RECORDREADER(split):
    for (docid, document) in enumerate(split):
      for (lineid, line) in enumerate(document.split('\n')):
        yield ("{}:{}".format(docid,lineid), line)

  split_size =  int(np.ceil(len(documents)/maps))
  for i in range(0, len(documents), split_size):
    yield RECORDREADER(documents[i:i+split_size])

def MAP(docId:str, line:str):
  for word in line.split(" "):
    yield (word, 1)

def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)

# try to set COMBINER=REDUCER and look at the number of values sent over the network
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

56 key-value pairs were sent over a network.


[(0, [('', 6)]),
 (1, [('a', 2), ('banana', 2), ('is', 18), ('it', 18), ('what', 10)])]

## TeraSort

In [78]:
import numpy as np

input_values = np.random.rand(30)
maps = 3
reducers = 2
min_value = 0.0
max_value = 1.0

def INPUTFORMAT():
  global maps

  def RECORDREADER(split):
    for value in split:
        yield (value, None)

  split_size =  int(np.ceil(len(input_values)/maps))
  for i in range(0, len(input_values), split_size):
    yield RECORDREADER(input_values[i:i+split_size])

def MAP(value:int, _):
  yield (value, None)

def PARTITIONER(key):
  global reducers
  global max_value
  global min_value
  bucket_size = (max_value-min_value)/reducers
  bucket_id = 0
  while((key>(bucket_id+1)*bucket_size) and ((bucket_id+1)*bucket_size<max_value)):
    bucket_id += 1
  return bucket_id

def REDUCE(value:int, _):
  yield (None,value)

partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None, PARTITIONER=PARTITIONER)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

30 key-value pairs were sent over a network.


[(0,
  [(None, np.float64(0.00847660511521453)),
   (None, np.float64(0.010073179224517448)),
   (None, np.float64(0.06533773476251303)),
   (None, np.float64(0.07329633554251269)),
   (None, np.float64(0.1032232938455443)),
   (None, np.float64(0.12995657094728974)),
   (None, np.float64(0.14463080004954054)),
   (None, np.float64(0.15836318365111646)),
   (None, np.float64(0.1608809343899268)),
   (None, np.float64(0.2691475655723964)),
   (None, np.float64(0.3091628768256621)),
   (None, np.float64(0.3765620319158569)),
   (None, np.float64(0.382245488157848)),
   (None, np.float64(0.38573863672353736)),
   (None, np.float64(0.41990172807439075)),
   (None, np.float64(0.4360803582210684)),
   (None, np.float64(0.46788153550577016)),
   (None, np.float64(0.47259718553537267)),
   (None, np.float64(0.485525980614372))]),
 (1,
  [(None, np.float64(0.5177297562419113)),
   (None, np.float64(0.5441628262023216)),
   (None, np.float64(0.5443647871557055)),
   (None, np.float64(0.584786946

# Упражнения
Упражнения взяты из Rajaraman A., Ullman J. D. Mining of massive datasets. – Cambridge University Press, 2011.


Для выполнения заданий переопределите функции RECORDREADER, MAP, REDUCE. Для модели распределённой системы может потребоваться переопределение функций PARTITION и COMBINER.

### Максимальное значение ряда

Разработайте MapReduce алгоритм, который находит максимальное число входного списка чисел.

In [79]:
input_numbers = [3, 7, 2, 9, 1, 8, 13, 4, 11]

def RECORDREADER():
  for value in input_numbers:
    yield (0, value)

def MAP(id:int, value:int):
  yield(id, value)

def REDUCE(id:int, array:list):
  yield(id, max(array))

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(0, 13)]

### Арифметическое среднее

Разработайте MapReduce алгоритм, который находит арифметическое среднее.

$$\overline{X} = \frac{1}{n}\sum_{i=0}^{n} x_i$$


In [80]:
input_numbers = [3, 7, 2, 9, 1, 8, 13, 4, 11]

def RECORDREADER():
  for value in input_numbers:
    yield (0, value)

def MAP(id:int, value:int):
  yield(id, value)

def REDUCE(id:int, array:list):
  yield(id, sum(array)/len(array))

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(0, 6.444444444444445)]

### GroupByKey на основе сортировки

Реализуйте groupByKey на основе сортировки, проверьте его работу на примерах

In [81]:
from collections import namedtuple

Person = namedtuple('Person', ['id', 'age', 'gender', 'contacts'])

data_persons = [
    Person(id=0, age=45, gender='male', contacts=15),
    Person(id=1, age=30, gender='female', contacts=310),
    Person(id=2, age=30, gender='female', contacts=450),
    Person(id=3, age=41, gender='male', contacts=60),
    Person(id=4, age=45, gender='female', contacts=220)
]

def RECORDREADER(key:str):
  for person in data_persons:
    yield (getattr(person, key), person)

def groupByKey(iterable):
  groups = {}
  for key, value in sorted(iterable, key=lambda x: x[0]):
    if key not in groups:
      groups[key] = []
    groups[key].append(value)
  return groups.items()

print("Group by 'age':", list(groupByKey(flatten(map(lambda x: MAP(*x), RECORDREADER("age"))))), "\n")
print("Group by 'gender':", list(groupByKey(flatten(map(lambda x: MAP(*x), RECORDREADER("gender"))))))

Group by 'age': [(30, [Person(id=1, age=30, gender='female', contacts=310), Person(id=2, age=30, gender='female', contacts=450)]), (41, [Person(id=3, age=41, gender='male', contacts=60)]), (45, [Person(id=0, age=45, gender='male', contacts=15), Person(id=4, age=45, gender='female', contacts=220)])] 

Group by 'gender': [('female', [Person(id=1, age=30, gender='female', contacts=310), Person(id=2, age=30, gender='female', contacts=450), Person(id=4, age=45, gender='female', contacts=220)]), ('male', [Person(id=0, age=45, gender='male', contacts=15), Person(id=3, age=41, gender='male', contacts=60)])]


### Drop duplicates (set construction, unique elements, distinct)

Реализуйте распределённую операцию исключения дубликатов

In [82]:
import numpy as np

data_with_duplicates = [
    Person(id=0, age=45, gender='male', contacts=15),
    Person(id=1, age=30, gender='female', contacts=310),
    Person(id=2, age=30, gender='female', contacts=450),
    Person(id=3, age=41, gender='male', contacts=60),
    Person(id=3, age=41, gender='male', contacts=60),
    Person(id=1, age=30, gender='female', contacts=310),
    Person(id=4, age=45, gender='female', contacts=220)
]

maps = 2
reducers = 2

def INPUTFORMAT():
  global maps

  def RECORDREADER(split):
    for person in split:
      yield (person.id, person)

  split_size = int(np.ceil(len(data_with_duplicates)/maps))
  for i in range(0, len(data_with_duplicates), split_size):
    yield RECORDREADER(data_with_duplicates[i:i+split_size])

def MAP(userId:int, person:Person):
  yield (userId, person)

def REDUCE(_, personsList:list):
  yield (list(set(personsList)))

partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

7 key-value pairs were sent over a network.


[(0,
  [[Person(id=0, age=45, gender='male', contacts=15)],
   [Person(id=2, age=30, gender='female', contacts=450)],
   [Person(id=4, age=45, gender='female', contacts=220)]]),
 (1,
  [[Person(id=1, age=30, gender='female', contacts=310)],
   [Person(id=3, age=41, gender='male', contacts=60)]])]

#Операторы реляционной алгебры
### Selection (Выборка)

**The Map Function**: Для  каждого кортежа $t \in R$ вычисляется истинность предиката $C$. В случае истины создаётся пара ключ-значение $(t, t)$. В паре ключ и значение одинаковы, равны $t$.

**The Reduce Function:** Роль функции Reduce выполняет функция идентичности, которая возвращает то же значение, что получила на вход.



In [83]:
data_for_selection = [
    Person(id=0, age=45, gender='male', contacts=15),
    Person(id=1, age=30, gender='female', contacts=310),
    Person(id=2, age=30, gender='female', contacts=450),
    Person(id=3, age=41, gender='male', contacts=60),
    Person(id=4, age=45, gender='female', contacts=220)
]

def RECORDREADER():
  for person in data_for_selection:
    yield (person.id, person)

def MAP(personId:int, person:Person):
  if person.age < 41:
    yield (person, person)

def REDUCE(person:Person, person_list:list):
  yield person

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[Person(id=1, age=30, gender='female', contacts=310),
 Person(id=2, age=30, gender='female', contacts=450)]

### Projection (Проекция)

Проекция на множество атрибутов $S$.

**The Map Function:** Для каждого кортежа $t \in R$ создайте кортеж $t′$, исключая  из $t$ те значения, атрибуты которых не принадлежат  $S$. Верните пару $(t′, t′)$.

**The Reduce Function:** Для каждого ключа $t′$, созданного любой Map задачей, вы получаете одну или несколько пар $(t′, t′)$. Reduce функция преобразует $(t′, [t′, t′, . . . , t′])$ в $(t′, t′)$, так, что для ключа $t′$ возвращается одна пара  $(t′, t′)$.

In [84]:
class PersonProjection(NamedTuple):
  id: int
  gender: str

data_for_projection = [
    Person(id=0, age=45, gender='male', contacts=15),
    Person(id=1, age=30, gender='female', contacts=310),
    Person(id=2, age=30, gender='female', contacts=450),
    Person(id=3, age=41, gender='male', contacts=60)
]

def RECORDREADER():
  for person in data_for_projection:
    yield (person.id, person)

def MAP(personId:int, person:Person):
  newPerson = PersonProjection(personId, person.gender)
  yield(newPerson, newPerson)

def REDUCE(key:PersonProjection, personsList:list):
  yield (key, key)

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(PersonProjection(id=0, gender='male'),
  PersonProjection(id=0, gender='male')),
 (PersonProjection(id=1, gender='female'),
  PersonProjection(id=1, gender='female')),
 (PersonProjection(id=2, gender='female'),
  PersonProjection(id=2, gender='female')),
 (PersonProjection(id=3, gender='male'),
  PersonProjection(id=3, gender='male'))]

### Union (Объединение)

**The Map Function:** Превратите каждый входной кортеж $t$ в пару ключ-значение $(t, t)$.

**The Reduce Function:** С каждым ключом $t$ будет ассоциировано одно или два значений. В обоих случаях создайте $(t, t)$ в качестве выходного значения.

In [85]:
group_a = [
    Person(id=1, age=30, gender='female', contacts=310),
    Person(id=2, age=30, gender='female', contacts=450),
]

group_b = [
    Person(id=3, age=41, gender='male', contacts=60),
    Person(id=4, age=45, gender='female', contacts=220),
    Person(id=1, age=30, gender='female', contacts=310),
]

data_union = [group_a, group_b]

def RECORDREADER():
  for group in data_union:
    for person in group:
      yield (person.id, person)

def MAP(personId:int, person:Person):
  yield(person, person)

def REDUCE(key:Person, personsList:list):
  yield (key, key)

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(Person(id=1, age=30, gender='female', contacts=310),
  Person(id=1, age=30, gender='female', contacts=310)),
 (Person(id=2, age=30, gender='female', contacts=450),
  Person(id=2, age=30, gender='female', contacts=450)),
 (Person(id=3, age=41, gender='male', contacts=60),
  Person(id=3, age=41, gender='male', contacts=60)),
 (Person(id=4, age=45, gender='female', contacts=220),
  Person(id=4, age=45, gender='female', contacts=220))]

### Intersection (Пересечение)

**The Map Function:** Превратите каждый кортеж $t$ в пары ключ-значение $(t, t)$.

**The Reduce Function:** Если для ключа $t$ есть список из двух элементов $[t, t]$ $-$ создайте пару $(t, t)$. Иначе, ничего не создавайте.

In [86]:
group_1 = [
    Person(id=0, age=45, gender='male', contacts=15),
    Person(id=1, age=30, gender='female', contacts=310),
    Person(id=2, age=30, gender='female', contacts=450),
]

group_2 = [
    Person(id=1, age=30, gender='female', contacts=310),
    Person(id=2, age=30, gender='female', contacts=450),
    Person(id=3, age=41, gender='male', contacts=60),
]

data_intersection = [group_1, group_2]

def RECORDREADER():
  for group in data_intersection:
    for person in group:
      yield (person.id, person)

def MAP(personId:int, person:Person):
  yield(person, person)

def REDUCE(key:Person, personsList:list):
  if len(personsList) == 2:
    yield (key, key)

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(Person(id=1, age=30, gender='female', contacts=310),
  Person(id=1, age=30, gender='female', contacts=310)),
 (Person(id=2, age=30, gender='female', contacts=450),
  Person(id=2, age=30, gender='female', contacts=450))]

### Difference (Разница)

**The Map Function:** Для кортежа $t \in R$, создайте пару $(t, R)$, и для кортежа $t \in S$, создайте пару $(t, S)$. Задумка заключается в том, чтобы значение пары было именем отношения $R$ or $S$, которому принадлежит кортеж (а лучше, единичный бит, по которому можно два отношения различить $R$ or $S$), а не весь набор атрибутов отношения.

**The Reduce Function:** Для каждого ключа $t$, если соответствующее значение является списком $[R]$, создайте пару $(t, t)$. В иных случаях не предпринимайте действий.

In [87]:
group_R = [
    Person(id=0, age=45, gender='male', contacts=15),
    Person(id=1, age=30, gender='female', contacts=310),
    Person(id=2, age=30, gender='female', contacts=450),
]

group_S = [
    Person(id=1, age=30, gender='female', contacts=310),
    Person(id=3, age=41, gender='male', contacts=60),
]

data_difference = [group_R, group_S]

def RECORDREADER():
  for index, group in enumerate(data_difference):
    for person in group:
      yield (index, person)

def MAP(groupId:int, person:Person):
  yield(person, groupId)

def REDUCE(key:Person, sourceLists:list):
  if len(sourceLists) == 1 and sourceLists[0] == 0:
    yield (key, key)

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(Person(id=0, age=45, gender='male', contacts=15),
  Person(id=0, age=45, gender='male', contacts=15)),
 (Person(id=2, age=30, gender='female', contacts=450),
  Person(id=2, age=30, gender='female', contacts=450))]

### Natural Join

**The Map Function:** Для каждого кортежа $(a, b)$ отношения $R$, создайте пару $(b,(R, a))$. Для каждого кортежа $(b, c)$ отношения $S$, создайте пару $(b,(S, c))$.

**The Reduce Function:** Каждый ключ $b$ будет асоциирован со списком пар, которые принимают форму либо $(R, a)$, либо $(S, c)$. Создайте все пары, одни, состоящие из  первого компонента $R$, а другие, из первого компонента $S$, то есть $(R, a)$ и $(S, c)$. На выходе вы получаете последовательность пар ключ-значение из списков ключей и значений. Ключ не нужен. Каждое значение, это тройка $(a, b, c)$ такая, что $(R, a)$ и $(S, c)$ это принадлежат входному списку значений.

In [88]:
class Employee(NamedTuple):
  emp_id: int
  dept_id: int
  name: str

class Department(NamedTuple):
  dept_id: int
  dept_name: str

employees = [
    Employee(emp_id=101, dept_id=10, name='Alice'),
    Employee(emp_id=102, dept_id=20, name='Bob'),
    Employee(emp_id=103, dept_id=10, name='Charlie'),
    Employee(emp_id=104, dept_id=30, name='Diana'),
]

departments = [
    Department(dept_id=10, dept_name='Engineering'),
    Department(dept_id=20, dept_name='Marketing'),
]

data_for_join = [employees, departments]

def RECORDREADER():
  for index, relation in enumerate(data_for_join):
    for item in relation:
      yield (index, item)

def MAP(relationId:int, tuple_data:NamedTuple):
  if relationId == 0:
    yield (tuple_data.dept_id, ('R', tuple_data.emp_id))
  else:
    yield (tuple_data.dept_id, ('S', tuple_data.dept_name))

def REDUCE(dept_id:int, values:list):
  r_values = [a for (rel, a) in values if rel == 'R']
  s_values = [c for (rel, c) in values if rel == 'S']
  for emp_id in r_values:
    for dept_name in s_values:
      yield (emp_id, dept_id, dept_name)

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(101, 10, 'Engineering'), (103, 10, 'Engineering'), (102, 20, 'Marketing')]

### Grouping and Aggregation (Группировка и аггрегация)

**The Map Function:** Для каждого кортежа $(a, b, c$) создайте пару $(a, b)$.

**The Reduce Function:** Ключ представляет ту или иную группу. Примение аггрегирующую операцию $\theta$ к списку значений $[b1, b2, . . . , bn]$ ассоциированных с ключом $a$. Возвращайте в выходной поток $(a, x)$, где $x$ результат применения  $\theta$ к списку. Например, если $\theta$ это $SUM$, тогда $x = b1 + b2 + · · · + bn$, а если $\theta$ is $MAX$, тогда $x$ это максимальное из значений $b1, b2, . . . , bn$.

In [89]:
class Sale(NamedTuple):
  product_id: int
  amount: float
  month: str

sales_data = [
    Sale(product_id=1, amount=150.0, month='Jan'),
    Sale(product_id=2, amount=200.0, month='Jan'),
    Sale(product_id=1, amount=75.0, month='Feb'),
    Sale(product_id=1, amount=100.0, month='Mar'),
    Sale(product_id=2, amount=300.0, month='Feb'),
    Sale(product_id=3, amount=50.0, month='Jan'),
]

def RECORDREADER():
  for sale in sales_data:
    yield (sale.product_id, sale)

def MAP(productId:int, sale:Sale):
  yield (productId, sale.amount)

def REDUCE(productId:int, amounts:list):
  yield (productId, max(amounts))

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(1, 150.0), (2, 300.0), (3, 50.0)]

#

### Matrix-Vector multiplication

Случай, когда вектор не помещается в памяти Map задачи


In [90]:
import numpy as np

matrix_mv = np.random.rand(4, 4)
vector_mv = np.array([2, 1, 3, 0])

maps = 2
reducers = 2

def INPUTFORMAT():
    global maps

    def RECORDREADER(split):
        for i in range(split.shape[0]):
            for j in range(split.shape[1]):
                yield ((i, j), (split[i, j], vector_mv[j], split.shape[1]))

    split_size = int(np.ceil(len(matrix_mv) / maps))

    for i in range(0, len(matrix_mv), split_size):
        yield RECORDREADER(matrix_mv[i: i + split_size])

def MAP(coordinates: tuple, values: tuple):
    i, j = coordinates
    matrix_value, vector_value, cols = values
    yield ((i, cols), matrix_value * vector_value)

def REDUCE(keys: tuple, products: list):
    i, cols = keys
    yield (i, sum(products))

partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

16 key-value pairs were sent over a network.


[(0, [(0, np.float64(7.110434036935416)), (1, np.float64(9.647040490702276))]),
 (1, [])]

## Matrix multiplication (Перемножение матриц)

Если у нас есть матрица $M$ с элементами $m_{ij}$ в строке $i$ и столбце $j$, и матрица $N$ с элементами $n_{jk}$ в строке $j$ и столбце $k$, тогда их произведение $P = MN$ есть матрица $P$ с элементами $p_{ik}$ в строке $i$ и столбце $k$, где

$$p_{ik} =\sum_{j} m_{ij}n_{jk}$$

Необходимым требованием является одинаковое количество столбцов в $M$ и строк в $N$, чтобы операция суммирования по  $j$ была осмысленной. Мы можем размышлять о матрице, как об отношении с тремя атрибутами: номер строки, номер столбца, само значение. Таким образом матрица $M$ предстваляется как отношение $ M(I, J, V )$, с кортежами $(i, j, m_{ij})$, и, аналогично, матрица $N$ представляется как отношение $N(J, K, W)$, с кортежами $(j, k, n_{jk})$. Так как большие матрицы как правило разреженные (большинство значений равно 0), и так как мы можем нулевыми значениями пренебречь (не хранить), такое реляционное представление достаточно эффективно для больших матриц. Однако, возможно, что координаты $i$, $j$, и $k$ неявно закодированы в смещение позиции элемента относительно начала файла, вместо явного хранения. Тогда, функция Map (или Reader) должна быть разработана таким образом, чтобы реконструировать компоненты $I$, $J$, и $K$ кортежей из смещения.

Произведение $MN$ это фактически join, за которым следуют группировка по ключу и аггрегация. Таким образом join отношений $M(I, J, V )$ и $N(J, K, W)$, имеющих общим только атрибут $J$, создаст кортежи $(i, j, k, v, w)$ из каждого кортежа $(i, j, v) \in M$ и кортежа $(j, k, w) \in N$. Такой 5 компонентный кортеж представляет пару элементов матрицы $(m_{ij} , n_{jk})$. Что нам хотелось бы получить на самом деле, это произведение этих элементов, то есть, 4 компонентный кортеж$(i, j, k, v \times w)$, так как он представляет произведение $m_{ij}n_{jk}$. Мы представляем отношение как результат одной MapReduce операции, в которой мы можем произвести группировку и аггрегацию, с $I$ и $K$  атрибутами, по которым идёт группировка, и суммой  $V \times W$.





In [91]:
# MapReduce model
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

Реализуйте перемножение матриц с использованием модельного кода MapReduce для одной машины в случае, когда одна матрица хранится в памяти, а другая генерируется RECORDREADER-ом.

In [92]:
import numpy as np
I = 2
J = 3
K = 4*10
small_mat = np.random.rand(I,J) # it is legal to access this from RECORDREADER, MAP, REDUCE
big_mat = np.random.rand(J,K)

def RECORDREADER():
  for j in range(big_mat.shape[0]):
    for k in range(big_mat.shape[1]):
      yield ((j,k), big_mat[j,k])


def MAP(k1, v1):
    (j, k) = k1
    w = v1
    for i in range(I):
        yield ((i, k), small_mat[i, j] * w)


def REDUCE(key, values):
    (i, k) = key
    yield (i, k), sum(values)

Проверьте своё решение

In [93]:
# CHECK THE SOLUTION
reference_solution = np.matmul(small_mat, big_mat)
solution = MapReduce(RECORDREADER, MAP, REDUCE)

def asmatrix(reduce_output):
  reduce_output = list(reduce_output)
  I = max(i for ((i,k), vw) in reduce_output)+1
  K = max(k for ((i,k), vw) in reduce_output)+1
  mat = np.empty(shape=(I,K))
  for ((i,k), vw) in reduce_output:
    mat[i,k] = vw
  return mat

np.allclose(reference_solution, asmatrix(solution)) # should return true

True

In [94]:
reduce_output = list(MapReduce(RECORDREADER, MAP, REDUCE))
max(i for ((i,k), vw) in reduce_output)

1

Реализуйте перемножение матриц  с использованием модельного кода MapReduce для одной машины в случае, когда обе матрицы генерируются в RECORDREADER. Например, сначала одна, а потом другая.

In [95]:
I = 3
J = 4
K = 5 * 10

mat_A = np.random.rand(I, J)
mat_B = np.random.rand(J, K)

def RECORDREADER():
  for i in range(mat_A.shape[0]):
    for j in range(mat_A.shape[1]):
      for k in range(mat_B.shape[1]):
        yield (((i, j), mat_A[i, j]), ((j, k), mat_B[j, k]))

def MAP(element_A, element_B):
  (coord_A, val_A) = element_A
  (coord_B, val_B) = element_B
  yield ((coord_A[0], coord_B[1]), val_A * val_B)

def REDUCE(key, values):
  (i, k) = key
  yield ((i, k), sum(values))

Проверьте своё решение

In [96]:
# Проверка
reference_solution_both = np.matmul(mat_A, mat_B)
solution_both = MapReduce(RECORDREADER, MAP, REDUCE)

def asmatrix(reduce_output):
  reduce_output = list(reduce_output)
  I_max = max(i for ((i, k), vw) in reduce_output) + 1
  K_max = max(k for ((i, k), vw) in reduce_output) + 1
  mat = np.empty(shape=(I_max, K_max))
  for ((i, k), vw) in reduce_output:
    mat[i, k] = vw
  return mat

np.allclose(reference_solution_both, asmatrix(solution_both))

True

Реализуйте перемножение матриц с использованием модельного кода MapReduce Distributed, когда каждая матрица генерируется в своём RECORDREADER.

In [97]:
I = 3
J = 4
K = 5*10

maps = 2
reducers = 3

small_mat = np.random.rand(I, J)
big_mat = np.random.rand(J, K)

def INPUTFORMAT():
  def RECORDREADER(key, split):
    mat = []
    for i in range(split.shape[0]):
      for j in range(split.shape[1]):
        mat.append(((key, i, j), split[i, j]))
    return mat

  yield RECORDREADER('S', small_mat)
  yield RECORDREADER('B', big_mat)


def MAP(k, v):
  (mat, i, j) = k
  w = v
  if mat == 'S':
    yield (j, (mat, i, w))
  else:
    yield (i, (mat, j, w))

def REDUCE(_, values):
  small = [v for v in values if v[0] == 'S']
  big = [v for v in values if v[0] == 'B']
  for s in small:
    for b in big:
      yield ((s[1], b[1]), s[2] * b[2])

def INPUT_MUL():
  for j in joined:
    yield j[1]

def MAP_MUL(k1, v1):
  yield (k1, v1)

def REDUCE_MUL(key, values):
  res_val = 0
  for v in values:
    res_val += v
  yield (key, res_val)

partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
joined = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]

mul_output = MapReduceDistributed(INPUT_MUL, MAP_MUL, REDUCE_MUL, COMBINER=None)
pre_result = [(partition_id, list(partition)) for (partition_id, partition) in mul_output]

solution = []
for p in pre_result:
    for v in p[1]:
        solution.append(v)

212 key-value pairs were sent over a network.
600 key-value pairs were sent over a network.


Проверьте своё решение

In [98]:
# Проверка
reference_solution = np.matmul(small_mat, big_mat)

def asmatrix(reduce_output):
  reduce_output = list(reduce_output)
  I = max(i for ((i,k), vw) in reduce_output)+1
  K = max(k for ((i,k), vw) in reduce_output)+1
  mat = np.empty(shape=(I,K))
  for ((i,k), vw) in reduce_output:
    mat[i,k] = vw
  return mat

np.allclose(reference_solution, asmatrix(solution))

True

Обобщите предыдущее решение на случай, когда каждая матрица генерируется несколькими RECORDREADER-ами, и проверьте его работоспособность. Будет ли работать решение, если RECORDREADER-ы будут генерировать случайное подмножество элементов матрицы?

In [124]:
I = 3
J = 4
K = 5*10

maps = 2
reducers = 3

small_mat = np.random.rand(I, J)
big_mat = np.random.rand(J, K)

def INPUTFORMAT():
    global maps
    def RECORDREADER(key, split):
        mat = []
        for i in range(split.shape[0]):
            for j in range(split.shape[1]):
                mat.append(((key, i, j), split[i, j]))
        return mat

    small = RECORDREADER('S', small_mat)
    split_size =  int(np.ceil(len(small)/maps))
    for i in range(0, len(small), split_size):
        yield small[i:i+split_size]

    big = RECORDREADER('B', big_mat)
    split_size =  int(np.ceil(len(big)/maps))
    for i in range(0, len(big), split_size):
      yield big[i:i+split_size]

def MAP(k, v):
    (mat, i, j) = k
    w = v
    if mat == 'S':
        yield (j, (mat, i, w))
    else:
        yield (i, (mat, j, w))

def REDUCE(key, values):
    small = [v for v in values if v[0] == 'S']
    big = [v for v in values if v[0] == 'B']
    for s in small:
        for b in big:
          yield ((s[1], b[1]), s[2] * b[2])

def INPUT_MUL():
    for j in joined:
        yield j[1]

def MAP_MUL(k1, v1):
    yield (k1, v1)

def REDUCE_MUL(key, values):
    res_val = 0
    for v in values:
      res_val += v
    yield (key, res_val)

partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
joined = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]

mul_output = MapReduceDistributed(INPUT_MUL, MAP_MUL, REDUCE_MUL, COMBINER=None)
pre_result = [(partition_id, list(partition)) for (partition_id, partition) in mul_output]

solution = []
for p in pre_result:
    for v in p[1]:
      solution.append(v)

212 key-value pairs were sent over a network.
600 key-value pairs were sent over a network.


Проверьте своё решение

In [127]:
# Проверка
reference_solution = np.matmul(small_mat, big_mat)

def asmatrix(reduce_output):
  reduce_output = list(reduce_output)
  I_max = max(i for ((i, k), vw) in reduce_output) + 1
  K_max = max(k for ((i, k), vw) in reduce_output) + 1
  mat = np.empty(shape=(I_max, K_max))
  for ((i, k), vw) in reduce_output:
    mat[i, k] = vw
  return mat

np.allclose(reference_solution, asmatrix(solution))

True

В функции MAP ключи формируются на основе координат элементов матрицы. При случайной выборке элементов существует риск нарушения соответствия между сгенерированным ключом и реальной позицией элемента в матрице, что приведет к некорректным результатам вычислений. Таким образом, корректность алгоритма напрямую зависит от точного учета пространственного расположения элементов.